## 建立神经网络 以Lenet5为例

mindspore.nn 提供了各种网络基础模块

导包

In [1]:
import numpy as np
import mindspore
import mindspore.nn as nn 
from mindspore import Tensor

## 定义模型类

mindspore的`Cell`类是构建所有网络的基类 也是网络的基本单元 

当需要构建神经网络的时候 需要集成Cell类

重写__init__方法 和 construct 方法

In [2]:
class LeNet5(nn.Cell):
    def __init__(self,num_class=10,num_channel = 1):
        # 初始化网络
        super(LeNet5,self).__init__()
        # 定义所需要的运算
        self.conv1 = nn.Conv2d(num_channel,6,5,pad_mode='valid') # 卷积
        self.conv2 = nn.Conv2d(6,16,5,pad_mode='valid')
        self.fc1 = nn.Dense(16*5*5,120) # 全连接层
        self.fc2 = nn.Dense(120,84)
        self.fc3 = nn.Dense(84,num_class)
        self.max_pool2d = nn.MaxPool2d(kernel_size=2,stride=2)# 最大池化-降采样
        self.relu = nn.ReLU() # 激活函数
        self.flatten = nn.Flatten()# flatten 扁平的意思=> 将原来的高维数组换成只有 一行 的数组 列数是之前的各维度之积
        
        # 定义网络构建函数
    def construct(self,x):
        # 构建前向网络
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x    

## 模型函数

In [3]:
# nn.Conv2d 卷积函数 提取特征
conv2d = nn.Conv2d(1,6,5,has_bias=False,weight_init='normal',pad_mode='valid') # valid 就是 0 padding
input_x= Tensor(np.ones([1,1,32,32]),mindspore.float32)

print(conv2d(input_x).shape)

(1, 6, 28, 28)


In [4]:
# nn.ReLU 非线性激活函数，学习复杂特征
relu = nn.ReLU()
input_x = Tensor(np.array([1,2,-6,-1,3]),mindspore.float16) # 所有的负数变为0 所有正数保留
output = relu(input_x)
print(output)

[1. 2. 0. 0. 3.]


In [5]:
# nn.MaxPool2d 降采样
max_pool2d = nn.MaxPool2d(kernel_size=2,stride=2)
input_x = Tensor(np.ones([1,6,28,28]),mindspore.float16)
print(max_pool2d(input_x).shape)

(1, 6, 14, 14)


In [6]:
# nn.Flatten 将16*5*5 转化为 1*400的连续数组
flatten = nn.Flatten()
input_x = Tensor(np.ones([1,16,5,5]),mindspore.float16)
# input_x = Tensor(np.array([1,16,5,5]),mindspore.float16) # 输出了意想不到的 (4,1)
output = flatten(input_x)
print(output.shape)

(1, 400)


In [7]:
# nn.Dense 全连接层 对矩阵进行线性变换
dense = nn.Dense(400,120,weight_init='normal') # 默认使用mindspore.float32
input_x = Tensor(np.ones([1,400]),mindspore.float32)
output = dense(input_x)
print(output.shape)


(1, 120)


## 模型参数
实例化内部的卷积层和全连接层后 

即具有权重和偏置

这些值会在后面的训练中优化

`nn.Cell`中使用`parameters_and_names()`方法访问所有参数

遍历并打印各层网络名字和属性


In [8]:
# 初始化超参数
model = LeNet5()
for m in model.parameters_and_names():
    print(m)

('conv1.weight', Parameter (name=conv1.weight, shape=(6, 1, 5, 5), dtype=Float32, requires_grad=True))
('conv2.weight', Parameter (name=conv2.weight, shape=(16, 6, 5, 5), dtype=Float32, requires_grad=True))
('fc1.weight', Parameter (name=fc1.weight, shape=(120, 400), dtype=Float32, requires_grad=True))
('fc1.bias', Parameter (name=fc1.bias, shape=(120,), dtype=Float32, requires_grad=True))
('fc2.weight', Parameter (name=fc2.weight, shape=(84, 120), dtype=Float32, requires_grad=True))
('fc2.bias', Parameter (name=fc2.bias, shape=(84,), dtype=Float32, requires_grad=True))
('fc3.weight', Parameter (name=fc3.weight, shape=(10, 84), dtype=Float32, requires_grad=True))
('fc3.bias', Parameter (name=fc3.bias, shape=(10,), dtype=Float32, requires_grad=True))
